# Libraries

In [70]:
import ast
import gensim
import gensim.corpora as corpora 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models
import scipy.cluster.hierarchy as shc

from collections import Counter
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans
from sklearn.metrics import silhouette_score
from sklearn.neighbors._nearest_centroid import NearestCentroid

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Data

In [2]:
file_path = "/Users/catherinepollack/Documents/dartmouth/research/aim3_facebook_covid19_obesity/data"
dat_obesity = "/211105_feature_matrix_bert.csv"
dat_health = "/211214_feature_matrix_bert_health_comparator.csv"
dat_nonhealth = "/211214_feature_matrix_bert_nonhealth_comparator.csv"

dat_obesity = pd.read_csv(str(file_path + dat_obesity))
dat_health = pd.read_csv(str(file_path + dat_health))
dat_nonhealth = pd.read_csv(str(file_path + dat_nonhealth))

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (84) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (97,98,99,100) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Feature Matrix Curation

## Obesity

In [3]:
#api whatsapp : without skip
#neg: compound
#WC: OtherP
#BERT 0 : BERT 767
dat_obesity_1 = dat_obesity.loc[:,"api whatsapp":"without skip"]
dat_obesity_2 = dat_obesity.loc[:,"neg":"compound"]
dat_obesity_3 = dat_obesity.loc[:,"WC":"OtherP"]
dat_obesity_4 = dat_obesity.loc[:,"BERT 0":"BERT 767"]
dat_obesity_fm = pd.concat([dat_obesity_1,
                            dat_obesity_2,
                            dat_obesity_3,
                            dat_obesity_4],
                            axis = 1)
dat_obesity_fm.head()
print(dat_obesity_fm.shape) #629960                           


(629960, 938)


# Clustering

- Randomly Sample and Run Hierarchical Clustering, then run KMeans and DBSCAN
- LDA
- Pick Features Based on Reddit Analysis

## Randomly Sample and Run Hierarchical Clustering

### Part 1. Sample and Run Initial Hierarchical Clustering

In [6]:
## Randomly sample 1% of the data frame
dat_obesity_fm_1 = dat_obesity_fm.sample(frac = 0.01,
                                          random_state = 110295)

In [ ]:
## Run hierarchical clustering - Note: takes a long time
plt.figure(figsize=(14, 7))
dend = shc.dendrogram(shc.linkage(dat_obesity_fm_1, 
                                  method='ward'))

### Part 2: Assign Labels to Sample and Identify Centroids

#### Identify Number of Clusters

In [9]:
Counter(dend['leaves_color_list']) #4 clusters

Counter({'C1': 2, 'C2': 4380, 'C3': 731, 'C4': 1047, 'C5': 140})

#### Clustering Sampled Data to Find Centroid

In [7]:
model_agg = AgglomerativeClustering(n_clusters = 4,
                                    affinity="euclidean",
                                    linkage = "ward")

yhat_agg = model_agg.fit_predict(dat_obesity_fm_1)

#### Finding Centroids

In [8]:
find_center = NearestCentroid()
find_center.fit(dat_obesity_fm_1, yhat_agg)
centoids = find_center.centroids_

### Part 3. Run K-Means and DBSCAN to Get Full Clustering

#### KMeans with Centers from Agglomerative Clustering

In [11]:
model_kmeans = KMeans(n_clusters = 4,
                      random_state = 110295,
                      init = centoids)
yhat_kmeans = model_kmeans.fit_predict(dat_obesity_fm)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


#### K-Means ++ with Same # of Clusters but No Centroids

In [12]:
model_kmeans = KMeans(n_clusters = 4,
                      random_state = 110295,
                      init = "k-means++")
yhat_kmeans_plus = model_kmeans.fit_predict(dat_obesity_fm)

#### DBSCAN on Cluster

## LDA Clustering

In [44]:
texts = [ast.literal_eval(x) for x in dat_obesity.tokens]

terms = [item for sublist in texts for item in sublist]
terms = list(set(terms)) #106269

In [65]:
# Create Dictionary
id2word = corpora.Dictionary(texts) 

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts] 

# number of topics
num_topics = 4

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
                                       
# Print the Keyword in the 4 topics
lda_model.print_topics()

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: prepare() missing 2 required positional arguments: 'vocab' and 'term_frequency'

In [72]:
vis = pyLDAvis.gensim_models.prepare(topic_model = lda_model, 
                       corpus = corpus, 
                       dictionary = id2word)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Obesity Data

In [ ]:
obesity_labels = pd.DataFrame({"agglomerate": yhat_agg,
                    "dbscan": yhat_db,
                    "birch": yhat_birch,
                    "kmeans": yhat_kmeans})

print("Silhouette Scores for Health Comparator")
print("Agglomerative")
print(silhouette_score(dat_obesity_fm, obesity_labels.agglomerate, metric='euclidean')) #0.358
print("DBSCAN")
print(silhouette_score(dat_obesity_fm, obesity_labels.dbscan, metric='euclidean')) #0.106
print("BIRCH")
print(silhouette_score(dat_obesity_fm, obesity_labels.birch, metric='euclidean')) #0.343
print("KMeans")
print(silhouette_score(dat_obesity_fm, obesity_labels.kmeans, metric='euclidean')) #0.241